# Matches_Predicao

Este notebook é dedicado à predição de dados da tabela **Matches** fornecida pela IBM, que contém dados detalhados sobre as partidas, incluindo informações como quantidade de gols nas partidas, cartões amarelos que têm no total, entre outros. O objetivo deste notebook é confirmar as hipóteses e prever mais informações com os dados dessa tabela.

## Objetivo

O objetivo deste notebook é fornecer uma predição de dados das partidas da série A do Campeonato Brasileiro, ajudando a identificar padrões e tendências que possam ser úteis para diversas aplicações, como previsões de resultados e desempenho dos times.

## Como Usar Este Notebook

1. **Configuração do Ambiente**:
   - **Google Colab**: No Google Colab, será necessário fazer o upload das tabelas para o Google Drive e montar o drive no notebook.
   - **Localmente**: Se for rodar o notebook localmente, é necessário baixar as tabelas e colocá-las no mesmo diretório do notebook ou ajustar os caminhos dos arquivos conforme necessário.

2. **Instalação de Dependências**:
   - Certifique-se de que todas as bibliotecas necessárias estão instaladas. Você pode instalar as dependências utilizando o seguinte comando:
     ```python
     !pip install -r requirements.txt
     ```

3. **Execução do Notebook**:
   - Antes de rodar esse notebook, execute por completo o notebook `matches_tratado.ipynb` para que sejam exportadas em suas células a
   respectiva tabela contendo os dados tratados, que serão utilizados neste notebook.
   - Siga as células de código sequencialmente para garantir que todas as etapas sejam executadas corretamente. Cada seção do notebook está organizada para facilitar a compreensão e a análise dos dados.

## Nesse Notebook Será Abordado

1. **Modelagem para o problema**:
   - Modelagem para o problema - proposta das features;
   - Organização dos dados - como os dados foram organizados;
   - Apresentar o modelo candidato - qual modelo estamos utilizando e para que;
   - Métricas relacionadas ao modelo - avaliação de desempenho.

# Dependências
Para rodar o notebook de forma local, é recomendado que inicie uma venv (ambiente virtual) e instale as dependências.

Se estiver utilizando o Google Colab, pule esta etapa.


# Instala as dependências

In [ ]:
# Instala as dependências
!pip install -r requirements.txt

# Importando bibliotecas

Aqui é importado as dependências necessárias para a executação do projeto.

In [167]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score, confusion_matrix, roc_curve, roc_auc_score, RocCurveDisplay, auc
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import label_binarize


# Carregando o Dataset
Carrega o CSV da tabela matches tratado para criar um dataframe.

In [168]:
df = pd.read_csv('../../notebooks/data/tratado/matches_tratado.csv')

# 1. Organização dos dados

Após diversos testes de proproções de conjuntos de dados, a combinação que trouxe uma maior acurácia para os modelos deste notebook foi o de 70% de dados para treinamento e 30% para dados de teste.

# 2. Modelagem para o Problema (Seleção das Principais Features)

Nesta etapa, utilizamos o **Random Forest** para identificar as principais variáveis (features) que influenciam diretamente o resultado das partidas de futebol. A escolha dessas features foi baseada em uma análise cuidadosa do que mais impacta o desempenho de uma equipe em uma partida de futebol, tanto do ponto de vista ofensivo quanto defensivo.

## Proposta de Features e Linha de Raciocínio

### Premissa da Modelagem:

A modelagem é baseada na premissa de que o resultado de uma partida de futebol é influenciado por um conjunto de fatores que descrevem o desempenho passado e as condições pré-jogo das equipes envolvidas. Com isso, selecionamos as seguintes features como preditoras principais do resultado de uma partida:
- **Características das equipes**: Fatores como os times jogando em casa e fora (`home_team_encoded` e `away_team_encoded`), além de seu desempenho prévio medido por estatísticas como `Pre-Match PPG (Home)` e `Pre-Match PPG (Away)` (pontos por jogo antes da partida) e o `xG` (gols esperados).
- **Estatísticas ofensivas e defensivas**: Variáveis como número de chutes a gol, chutes no alvo, posse de bola e número de escanteios refletem o controle do time sobre o jogo e suas chances de marcar gols ou evitar que o adversário marque.
- **Faltas e Cartões**: Disciplinas das equipes também são consideradas, com indicadores como `home_team_yellow_cards`, `away_team_red_cards`, que podem impactar o equilíbrio do jogo se um time estiver desfalcado ou punido.

### Proposta das Features:

#### Variáveis Utilizadas:
Abaixo está a lista completa de features selecionadas para o modelo de previsão:
- **Identificação das Equipes**: `home_team_encoded`, `away_team_encoded`.
- **Desempenho Pré-Jogo**: `Pre-Match PPG (Home)`, `Pre-Match PPG (Away)`, `home_ppg`, `away_ppg`, `Home Team Pre-Match xG`, `Away Team Pre-Match xG`, `team_a_xg`, `team_b_xg`.
- **Estatísticas do Jogo**:
  - Ofensivas: `home_team_shots`, `away_team_shots`, `home_team_shots_on_target`, `away_team_shots_on_target`, `home_team_corner_count`, `away_team_corner_count`.
  - Defensivas: `home_team_fouls`, `away_team_fouls`.
- **Faltas e Cartões**: `home_team_yellow_cards`, `away_team_yellow_cards`, `home_team_red_cards`, `away_team_red_cards`.
- **Posse de Bola**: `home_team_possession`, `away_team_possession`.
- **Estatísticas Gerais**: `average_goals_per_match_pre_match`, `btts_percentage_pre_match` (ambos os times marcam), `average_corners_per_match_pre_match`, `average_cards_per_match_pre_match`.

### Explicação do Raciocínio:

1. **Desempenho Pré-Jogo**: Features como o `Pre-Match PPG` e `xG` são fundamentais para capturar a forma dos times antes da partida. Times com mais pontos por jogo e um alto valor de xG são mais propensos a manter bons desempenhos em jogos futuros.
   
2. **Estatísticas do Jogo**: O número de chutes e escanteios mostra como as equipes criam chances durante a partida, enquanto as faltas e cartões refletem o controle emocional e tático. Times mais ofensivos e disciplinados tendem a ganhar mais partidas.

3. **Cartões e Faltas**: Faltas e cartões são indicadores importantes da disciplina e da capacidade de manter o ritmo do jogo. Times com muitos cartões podem estar em desvantagem durante a partida.

4. **Posse de Bola**: Times com alta posse de bola geralmente controlam o jogo e têm maior probabilidade de vencer, pois criam mais oportunidades de gol e limitam as chances do adversário.

### Normalização das Variáveis:
Além da seleção cuidadosa das features, utilizamos o método **StandardScaler** para normalizar as variáveis numéricas. Isso foi essencial para garantir que todas as variáveis tenham a mesma escala, evitando que variáveis com magnitudes diferentes (como número de chutes vs. porcentagem de posse de bola) influenciem o modelo de forma desbalanceada.


In [ ]:
# Criando a coluna de resultado
df['result'] = np.where(df['home_team_goal_count'] > df['away_team_goal_count'], 2,
                        np.where(df['home_team_goal_count'] < df['away_team_goal_count'], 1, 0))

# Definindo a variável alvo e os preditores
X = df[['home_team_encoded', 'away_team_encoded', 'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg',
        'home_team_goal_count', 'away_team_goal_count', 'home_team_corner_count', 'away_team_corner_count',
        'home_team_yellow_cards', 'home_team_red_cards', 'away_team_yellow_cards', 'away_team_red_cards',
        'home_team_shots', 'away_team_shots', 'home_team_shots_on_target', 'away_team_shots_on_target',
        'home_team_shots_off_target', 'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
        'home_team_possession', 'away_team_possession', 'Home Team Pre-Match xG', 'Away Team Pre-Match xG',
        'team_a_xg', 'team_b_xg', 'average_goals_per_match_pre_match', 'btts_percentage_pre_match',
        'average_corners_per_match_pre_match', 'average_cards_per_match_pre_match']]
y = df['result']

# Imputação de valores faltantes
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Padronização dos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Criando e treinando o modelo de Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = rf.predict(X_test)

# Função para prever o resultado do jogo entre dois times
def predict_match_result(home_team_encoded, away_team_encoded):
    global imputer, scaler, rf

    # Seleciona as características do jogo a partir dos IDs dos times
    match_data = df[(df['home_team_encoded'] == home_team_encoded) & (df['away_team_encoded'] == away_team_encoded)].copy()

    # Se o DataFrame estiver vazio, retorna uma mensagem de erro
    if match_data.empty:
        return "Jogo não encontrado no conjunto de dados."

    # Realinhar as colunas para que correspondam ao conjunto de dados original usado para treinar o modelo
    match_data = match_data.reindex(columns=X.columns, fill_value=0)

    # Imputação e padronização dos dados
    match_data_imputed = imputer.transform(match_data)
    match_data_scaled = scaler.transform(match_data_imputed)

    # Prevendo o resultado do jogo
    prediction = rf.predict(match_data_scaled)[0]

    if prediction == 2:
        result = "Vitória do time da casa"
    elif prediction == 1:
        result = "Vitória do time visitante"
    else:
        result = "Empate"

    return result

# Exemplo de uso
home_team_id = 10
away_team_id = 13
print(predict_match_result(home_team_id, away_team_id))

# Avaliando o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))
print("Matriz de confusão\n", confusion_matrix(y_pred, y_test) )


#### Importância das Features:
Após treinar o modelo, podemos analisar a importância relativa de cada uma das variáveis preditoras. O modelo de Random Forest possui a capacidade de indicar quais features mais contribuíram para a decisão de cada árvore de decisão.

In [ ]:
# Verifica a importância das features do modelo treinado
importances = rf.feature_importances_

# Lista dos nomes das features
feature_names = [
    'home_team_encoded', 'away_team_encoded', 'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg',
    'home_team_goal_count', 'away_team_goal_count', 'home_team_corner_count', 'away_team_corner_count',
    'home_team_yellow_cards', 'home_team_red_cards', 'away_team_yellow_cards', 'away_team_red_cards',
    'home_team_shots', 'away_team_shots', 'home_team_shots_on_target', 'away_team_shots_on_target',
    'home_team_shots_off_target', 'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
    'home_team_possession', 'away_team_possession', 'Home Team Pre-Match xG', 'Away Team Pre-Match xG',
    'team_a_xg', 'team_b_xg', 'average_goals_per_match_pre_match', 'btts_percentage_pre_match',
    'average_corners_per_match_pre_match', 'average_cards_per_match_pre_match'
]

# Cria um DataFrame para organizar as importâncias das features
feature_importances = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# Ordena as features pela importância
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Exibe as 10 principais features
print("Principais Features:")
print(feature_importances.head(10))

# Visualiza as importâncias das features
plt.figure(figsize=(10, 6))
plt.title('Importância das Principais Features')
plt.barh(feature_importances['Feature'].head(10), feature_importances['Importance'].head(10))
plt.gca().invert_yaxis()  # Inverte o eixo y para que a feature mais importante apareça no topo
plt.xlabel('Importância')
plt.show()

# 3. Primeiro Modelo Candidato

### 3.1 Hipótese do Modelo: Qual a probabilidade de um time ganhar com certas variáveis.

##### Premissa Principal:
O objetivo de previsão desse modelo é que o desempenho passado de um time, medido por diversas métricas estatísticas (como posse de bola, número de chutes a gol, cartões, etc.), pode prever a probabilidade de um time vencer em uma partida futura. O modelo utiliza uma combinação de variáveis pré-jogo e dados históricos dos times para calcular essas probabilidades.

##### Justificativa da Premissa:
1. **Posse de bola e estatísticas de ataque e defesa** são fortes indicativos de domínio em uma partida. Times que têm alta posse de bola e boa eficiência nos chutes a gol, geralmente têm uma maior chance de vitória.
2. **Cartões, faltas e escanteios** influenciam diretamente o controle da partida e as oportunidades de gol. Uma defesa que comete muitas faltas ou recebe cartões pode ser vulnerável, enquanto escanteios e chances ofensivas contribuem para as probabilidades de marcar gols.
3. **Desempenho médio pré-jogo (PPG e xG)**: O número de pontos por jogo e o desempenho esperado de gols (xG) são métricas que oferecem uma visão do histórico do time e indicam sua capacidade de manter o desempenho ou superá-lo em partidas futuras.

#### Abordagem Técnica:

O modelo de **Random Forest** foi escolhido para capturar relações complexas entre as diversas variáveis. Ele é particularmente adequado para problemas com várias características preditivas, como o caso de previsão de resultados de futebol, onde a posse de bola, cartões, número de chutes, e outras estatísticas têm pesos diferentes dependendo do contexto.

- **Random Forest** constrói múltiplas árvores de decisão, onde cada árvore faz previsões baseadas em subconjuntos diferentes das variáveis. O resultado final é uma agregação das previsões dessas árvores, o que ajuda a melhorar a precisão do modelo e minimiza erros causados por outliers.
- O modelo foi treinado com um conjunto de dados que contém estatísticas de partidas anteriores e diversas métricas de desempenho dos times.

#### Estrutura do Modelo:

- **Variáveis Preditoras**:
  - Estatísticas da equipe da casa e da equipe visitante, incluindo posse de bola, número de chutes a gol, cartões amarelos e vermelhos, escanteios, xG (gols esperados) e faltas cometidas.
  - Dados pré-jogo como Pre-Match PPG (pontos por jogo) e Pre-Match xG (gols esperados).
- **Variável Alvo**: Resultado da partida (vitória da casa, empate ou vitória do visitante).

#### Previsão Ajustada:

O modelo prevê as probabilidades de três possíveis resultados para a partida:
- **Vitória da equipe da casa**
- **Empate**
- **Vitória da equipe visitante**

Além de fornecer as probabilidades em porcentagem, as previsões são categorizadas em níveis de probabilidade, utilizando a função `categorize_win_probability`, que classifica as chances de vitória em:
- Baixíssimas chances de ganhar
- Baixas chances de ganhar
- Médias chances de ganhar
- Muitas chances de ganhar
- Grandes chances de ganhar

Essa categorização ajuda a fornecer uma interpretação mais qualitativa das previsões feitas pelo modelo.

#### Exemplo de Funcionamento:

O modelo é capaz de prever resultados de partidas futuras, mesmo que os dados dessas partidas não estejam no conjunto de dados de treino. Nesse caso, ele utiliza **médias históricas** dos times para fazer previsões, garantindo que ainda possa fornecer uma probabilidade estimada, com base no desempenho geral dos times.

##### Exemplo de Previsão:
Para um confronto entre o time A e o time B, a saída do modelo pode ser:

- Probabilidade de vitória da casa (%): 75.0
- Chance de vitória da casa: Muitas chances de ganhar
- Probabilidade de empate (%): 15.0
- Probabilidade de vitória do visitante (%): 10.0
- Chance de vitória do visitante: Baixíssimas chances de ganhar


#### 3.2 Criando o Modelo

#### Criando a Coluna de Resultado e Definindo Variáveis Preditoras

**Explicação**:
- `df['result']`: Estamos criando uma nova coluna chamada result que representa o resultado da partida. A classe 2 indica vitória da casa, 1 vitória do visitante, e 0 indica empate.

- `Variáveis preditoras (X) e alvo (y)`: As variáveis preditoras são características como posse de bola, chutes a gol, cartões, etc., que são usadas para treinar o modelo. A variável alvo y é o resultado da partida.



In [171]:
# Criando a coluna de resultado
df['result'] = np.where(df['home_team_goal_count'] > df['away_team_goal_count'], 2, # Vitória da casa
                        np.where(df['home_team_goal_count'] < df['away_team_goal_count'], 1, # Vitória do visitante
                                 0)) # Derrota

# Definindo a variável alvo e os preditores
y = df['result']
X = df[['home_team_encoded', 'away_team_encoded', 'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg',
        'home_team_goal_count', 'away_team_goal_count', 'home_team_corner_count', 'away_team_corner_count',
        'home_team_yellow_cards', 'home_team_red_cards', 'away_team_yellow_cards', 'away_team_red_cards',
        'home_team_shots', 'away_team_shots', 'home_team_shots_on_target', 'away_team_shots_on_target',
        'home_team_shots_off_target', 'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
        'home_team_possession', 'away_team_possession', 'Home Team Pre-Match xG', 'Away Team Pre-Match xG',
        'team_a_xg', 'team_b_xg', 'average_goals_per_match_pre_match', 'btts_percentage_pre_match',
        'average_corners_per_match_pre_match', 'average_cards_per_match_pre_match']]


#### Imputação de Valores Faltantes, Padronização e Treinamento do Modelo

**Explicação:**

- `Imputação de valores faltantes:` Preenchemos os valores ausentes nas variáveis preditoras com a média da respectiva variável, garantindo que o modelo não falhe devido a dados incompletos.
- `Padronização dos dados:` Para evitar que as variáveis com escalas diferentes impactem negativamente o modelo, os dados são padronizados usando StandardScaler.
- `Divisão treino/teste:` O conjunto de dados é dividido em 70% para treino e 30% para teste, garantindo uma avaliação justa do modelo.
- `Random Forest:` Treinamos o modelo Random Forest com 100 árvores de decisão para fazer previsões sobre o resultado das partidas.
Avaliação: As métricas de acurácia, acurácia balanceada, e a matriz de confusão nos dão uma visão do desempenho do modelo no conjunto de teste.

In [ ]:
# Imputação de valores faltantes
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Padronização dos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Criando e treinando o modelo de Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Avaliando o modelo
y_pred = rf.predict(X_test)
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Acurácia Balanceada:", balanced_accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))
print("Matrix de confusão: \n", confusion_matrix(y_test, y_pred))


## Hiperparâmetros

Os hiperparâmetros são parâmetros ajustados antes do treinamento do modelo e desempenham um papel crucial no desempenho do mesmo. No caso do **Random Forest**, alguns dos hiperparâmetros mais importantes e suas funções são:

### n_estimators (número de estimadores):

Define o número de árvores na floresta. Um número maior de estimadores pode melhorar a performance do modelo, mas aumenta o tempo de treinamento e a complexidade computacional.

### max_depth (profundidade máxima):

Controla a profundidade máxima das árvores. Limitar a profundidade pode ajudar a evitar overfitting, principalmente em modelos complexos. `None` permite que nós sejam expandidos até que todas as folhas contenham menos do que min_samples_split amostras.

### min_samples_split (mínimo de amostras para dividir um nó):

Indica o número mínimo de amostras necessárias para dividir um nó interno. Valores mais altos previnem que a árvore se divida com poucos dados, ajudando a reduzir overfitting.

### min_samples_leaf (mínimo de amostras por folha):

Especifica o número mínimo de amostras que cada folha (nó terminal) deve conter. Ajustar esse hiperparâmetro é importante para reduzir o overfitting em dados ruidosos.

### bootstrap (amostragem com reposição):

Se `True`, as árvores são treinadas usando amostras com reposição, o que aumenta a diversidade entre as árvores e, em muitos casos, melhora a performance do modelo. Se `False`, as árvores são treinadas sem reposição, o que pode ser útil em certos contextos.

## GridSearchCV para ajuste de Hiperparâmetros:

O **GridSearchCV** é uma técnica que automatiza a busca pelos melhores hiperparâmetros, testando diferentes combinações de valores e avaliando o desempenho para selecionar a melhor configuração. Isso otimiza o modelo, maximizando a acurácia e reduzindo o risco de overfitting.

### Hiperparâmetros ajustados:

- `n_estimators`: [100, 200, 300]
- `max_depth`: [None, 10, 20, 30]
- `min_samples_split`: [2, 5, 10]
- `min_samples_leaf`: [1, 2, 4]
- `bootstrap`: [True, False]




In [ ]:
# Definindo os hiperparâmetros a serem otimizados
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Criando o modelo de Random Forest
rf = RandomForestClassifier(random_state=42)

# Aplicando o GridSearchCV para otimização dos hiperparâmetros
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Treinando o modelo com a otimização
grid_search.fit(X_train, y_train)

# Melhor combinação de hiperparâmetros
print("Melhores hiperparâmetros:", grid_search.best_params_)

# Atribuir o modelo otimizado à variável rf
rf = grid_search.best_estimator_

# Avaliando o modelo otimizado
y_pred = rf.predict(X_test)
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Acurácia Balanceada:", balanced_accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))
print("Matriz de confusão: \n", confusion_matrix(y_test, y_pred))



## Validação Cruzada
A **validação cruzada** é uma técnica fundamental para avaliar o desempenho de modelos de aprendizado de máquina de forma mais confiável. Em vez de treinar e testar o modelo apenas uma vez com um único conjunto de divisão de dados, a validação cruzada permite que o modelo seja testado múltiplas vezes em diferentes subconjuntos do conjunto de dados, garantindo que os resultados sejam menos dependentes de uma única divisão dos dados.

Neste projeto, utilizamos a validação cruzada com **5 folds estratificados**:
- O conjunto de dados é dividido em 5 partes (folds) de forma que a proporção das classes seja mantida em cada fold (estratificação).
- Em cada iteração, o modelo é treinado em 4 folds e testado no fold restante.
- O processo é repetido 5 vezes, trocando os folds de treinamento e teste, e ao final, é calculada a média das métricas de desempenho.


In [ ]:
# Configurando a validação cruzada com 5 folds estratificados
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Realizando a validação cruzada e obtendo as previsões de probabilidade
cv_scores = cross_val_score(rf, X, y, cv=cv, scoring='accuracy')
cv_predictions = cross_val_predict(rf, X, y, cv=cv, method='predict_proba')

# Média da acurácia da validação cruzada
print(f"Acurácia média da validação cruzada: {cv_scores.mean():.4f}")

## Curva ROC e AUC para o Modelo Random Forest
A **Curva ROC (Receiver Operating Characteristic)** é uma ferramenta gráfica que ilustra a capacidade de um modelo de classificação binária em distinguir entre classes positivas e negativas. A curva traça a relação entre a **Taxa de Verdadeiros Positivos (TPR)** e a **Taxa de Falsos Positivos (FPR)** para diferentes limiares de decisão.

O **AUC (Area Under the Curve)** mede a área sob a Curva ROC, fornecendo uma única métrica que reflete a capacidade do modelo de classificar corretamente as amostras. Um AUC próximo de 1 indica um modelo excelente, enquanto um AUC de 0.5 sugere que o modelo não é melhor que uma escolha aleatória.



In [ ]:
# Calculando a ROC e AUC para o modelo
# Para problemas de multiclassificação, é necessário binarizar o resultado para a curva ROC
# Vamos considerar a classe positiva como a vitória da casa, por exemplo (index 2)
y_true = y  # Labels verdadeiras
y_probs = cv_predictions[:, 2]  # Probabilidades da classe positiva (ex: vitória da casa)

# Calculando a curva ROC e o valor de AUC
fpr, tpr, _ = roc_curve(y_true, y_probs, pos_label=2)  # pos_label depende da classe positiva que está sendo analisada
roc_auc = auc(fpr, tpr)

# Plotando a curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Linha da chance (45 graus)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid()
plt.show()

# Exibindo o valor AUC da curva ROC
print(f"AUC da Curva ROC: {roc_auc:.4f}")

### Resultados

Após gerar a Curva ROC e calcular o AUC, os seguintes resultados foram obtidos:

- **Curva ROC**: A curva mostra a taxa de verdadeiros positivos (TPR) em função da taxa de falsos positivos (FPR) para diferentes limiares de decisão. O formato da curva fornece uma visão sobre o desempenho do modelo Random Forest em discriminar entre classes positivas e negativas.

- **AUC (Area Under the Curve)**: O valor calculado foi de **0.92**, indicando que o modelo possui uma excelente capacidade de classificação. Um AUC próximo de 1 sugere que o modelo é muito eficaz em distinguir corretamente entre as classes.

### Conclusão

A avaliação do modelo Random Forest usando a Curva ROC e AUC mostrou que o modelo tem um desempenho robusto na tarefa de classificação:

- **Desempenho de Classificação**: A curva ROC, que se aproxima do canto superior esquerdo do gráfico, demonstra que o modelo possui uma alta taxa de verdadeiros positivos em relação aos falsos positivos, o que é ideal para um classificador binário.

- **Qualidade do Modelo**: Com um AUC de **0.92**, o modelo demonstra excelente precisão e habilidade em classificar corretamente as instâncias. Este resultado sugere que o modelo Random Forest é adequado para ser utilizado no problema em questão, dado o seu alto poder discriminativo.

## Explicabilidade do Modelo Preditivo com LIME

O modelo preditivo utilizado neste projeto foi explicado usando a ferramenta **LIME (Local Interpretable Model-Agnostic Explanations)**, que fornece uma interpretação local do impacto de cada variável em uma previsão específica.

In [ ]:
import lime
import lime.lime_tabular

# Instanciando o objeto explicador do LIME
explainer_lime = lime.lime_tabular.LimeTabularExplainer(X_train,
                                                         feature_names=X.columns.tolist(),
                                                         class_names=['Derrota', 'Vitória do Visitante', 'Vitória da Casa'],
                                                         mode='classification')

# Selecionando uma amostra do conjunto de teste para explicar
i = 0  # Índice da amostra
exp_lime = explainer_lime.explain_instance(X_test[i], rf.predict_proba)

# Mostrando a explicação no notebook
exp_lime.show_in_notebook(show_table=True)



## Probabilidades da Previsão

As probabilidades previstas pelo modelo para o resultado da partida são as seguintes:

- **Probabilidade de Derrota (Empate)**: `0.28` (28%)
- **Probabilidade de Vitória do Visitante**: `0.46` (46%)
- **Probabilidade de Vitória da Casa**: `0.26` (26%)

O resultado previsto pelo modelo foi **Vitória do Visitante**.

## Principais Variáveis que Influenciaram a Previsão

As seguintes variáveis tiveram os maiores impactos na previsão do modelo, indicando os valores de cada feature para a partida e seu respectivo peso na previsão:

| **Variável**                          | **Valor**      | **Importância** |
|---------------------------------------|----------------|-----------------|
| home_team_goal_count                  | -0.37          | 0.19            |
| away_team_goal_count                  | 2.30           | 0.18            |
| home_ppg                              | -0.86          | 0.05            |
| average_cards_per_match_pre_match     | -0.52          | 0.02            |
| away_team_shots_off_target            | -1.37          | 0.02            |
| away_team_corner_count                | -1.06          | 0.02            |
| home_team_red_cards                   | -0.43          | 0.01            |
| home_team_possession                  | 0.10           | 0.01            |
| home_team_shots_off_target            | 1.30           | 0.01            |
| average_corners_per_match_pre_match   | 0.33           | 0.01            |

## Interpretação do Resultado

- **Vitória do Visitante** foi prevista como o resultado mais provável com uma probabilidade de 46%.
- A variável que mais influenciou o resultado foi o número de gols do **time visitante** (`away_team_goal_count`), seguido pelos gols do **time da casa** (`home_team_goal_count`).
- Outras variáveis importantes incluem a **média de pontos por jogo do time da casa** (`home_ppg`) e a **média de cartões por jogo antes da partida** (`average_cards_per_match_pre_match`).

## Conclusão

A análise com LIME permitiu entender que o número de gols do time visitante teve o maior impacto na previsão do modelo, seguido por outras variáveis relacionadas ao desempenho das equipes em jogos anteriores, como a média de pontos por jogo e o número de chutes a gol. O modelo identificou que, com base nesses fatores, a **Vitória do Visitante** era o resultado mais provável para a partida analisada.


#### Função para Categorizar a Probabilidade de Vitória
**Explicação:**
- Função de categorização: Esta função transforma as probabilidades numéricas de vitória em descrições qualitativas, ajudando a tornar os resultados mais interpretáveis para os usuários finais.

In [177]:
# Função para categorizar a probabilidade de vitória
def categorize_win_probability(prob):
    if prob < 0.2:
        return "Baixíssimas chances de ganhar"
    elif 0.2 <= prob < 0.4:
        return "Baixas chances de ganhar"
    elif 0.4 <= prob < 0.6:
        return "Médias chances de ganhar"
    elif 0.6 <= prob < 0.8:
        return "Muitas chances de ganhar"
    else:
        return "Grandes chances de ganhar"

#### Função de Previsão com Médias Históricas e Categorias de Probabilidade

**Explicação:**
- `Previsão com médias históricas:` Quando uma partida não é encontrada no conjunto de dados, o modelo utiliza as médias das estatísticas dos times para preencher as variáveis. Isso permite que o modelo faça previsões para jogos futuros ou não registrados.
- `Imputação e padronização:` Mesmo para partidas simuladas com médias, as etapas de imputação e padronização são aplicadas para garantir a consistência com o treinamento.
- `Classificação das probabilidades:` A função `categorize_win_probability` é usada para traduzir as probabilidades em categorias qualitativas (por exemplo, "Muitas chances de ganhar").

In [178]:
# Criar mapeamento dos times e IDs
home_team_map = dict(zip(df['home_team_encoded'], df['home_team_name']))
away_team_map = dict(zip(df['away_team_encoded'], df['away_team_name']))

# Função para obter o nome do time a partir do ID
def get_team_name(team_id, home=True):
    if home:
        return home_team_map.get(team_id, "Time não encontrado")
    else:
        return away_team_map.get(team_id, "Time não encontrado")

# Função para converter nome do time para o ID correspondente
def get_team_id(team_input, home=True):
    if isinstance(team_input, str):  # Se o input for uma string (nome do time)
        if home:
            return df[df['home_team_name'] == team_input]['home_team_encoded'].iloc[0]
        else:
            return df[df['away_team_name'] == team_input]['away_team_encoded'].iloc[0]
    else:
        return team_input  # Se já for um ID, retorna diretamente

# Função para prever o resultado de uma partida com suporte para ID ou nome de time
def predict_match_result_with_averages(home_team_input, away_team_input):
    global imputer, scaler, rf

    # Converter input para IDs se necessário
    home_team_encoded = get_team_id(home_team_input, home=True)
    away_team_encoded = get_team_id(away_team_input, home=False)

    # Obter os nomes dos times a partir dos IDs
    home_team_name = get_team_name(home_team_encoded, home=True)
    away_team_name = get_team_name(away_team_encoded, home=False)

    print(f"Previsão para o jogo: {home_team_name} vs {away_team_name}")

    # Verificar se a partida está no dataset
    match_data = df[(df['home_team_encoded'] == home_team_encoded) & (df['away_team_encoded'] == away_team_encoded)].copy()

    if match_data.empty:
        print("Jogo não encontrado no conjunto de dados. Usando médias históricas dos times.")

        # Calcular médias das estatísticas numéricas dos times para criar uma nova entrada
        home_team_stats = df[df['home_team_encoded'] == home_team_encoded].select_dtypes(include=[np.number]).mean()
        away_team_stats = df[df['away_team_encoded'] == away_team_encoded].select_dtypes(include=[np.number]).mean()

        # Criar uma nova entrada com as estatísticas médias
        match_data = pd.DataFrame({
            'Pre-Match PPG (Home)': [home_team_stats['Pre-Match PPG (Home)']],
            'Pre-Match PPG (Away)': [away_team_stats['Pre-Match PPG (Away)']],
            'home_ppg': [home_team_stats['home_ppg']],
            'away_ppg': [away_team_stats['away_ppg']],
            'home_team_corner_count': [home_team_stats['home_team_corner_count']],
            'away_team_corner_count': [away_team_stats['away_team_corner_count']],
            'home_team_yellow_cards': [home_team_stats['home_team_yellow_cards']],
            'home_team_red_cards': [home_team_stats['home_team_red_cards']],
            'away_team_yellow_cards': [away_team_stats['away_team_yellow_cards']],
            'away_team_red_cards': [away_team_stats['away_team_red_cards']],
            'home_team_shots': [home_team_stats['home_team_shots']],
            'away_team_shots': [away_team_stats['away_team_shots']],
            'home_team_shots_on_target': [home_team_stats['home_team_shots_on_target']],
            'away_team_shots_on_target': [away_team_stats['away_team_shots_on_target']],
            'home_team_shots_off_target': [home_team_stats['home_team_shots_off_target']],
            'away_team_shots_off_target': [away_team_stats['away_team_shots_off_target']],
            'home_team_fouls': [home_team_stats['home_team_fouls']],
            'away_team_fouls': [away_team_stats['away_team_fouls']],
            'home_team_possession': [home_team_stats['home_team_possession']],
            'away_team_possession': [away_team_stats['away_team_possession']],
            'Home Team Pre-Match xG': [home_team_stats['Home Team Pre-Match xG']],
            'Away Team Pre-Match xG': [away_team_stats['Away Team Pre-Match xG']],
            'team_a_xg': [home_team_stats['team_a_xg']],
            'team_b_xg': [away_team_stats['team_b_xg']],
            'average_goals_per_match_pre_match': [(home_team_stats['average_goals_per_match_pre_match'] + away_team_stats['average_goals_per_match_pre_match']) / 2],
            'btts_percentage_pre_match': [(home_team_stats['btts_percentage_pre_match'] + away_team_stats['btts_percentage_pre_match']) / 2],
            'average_corners_per_match_pre_match': [(home_team_stats['average_corners_per_match_pre_match'] + away_team_stats['average_corners_per_match_pre_match']) / 2],
            'average_cards_per_match_pre_match': [(home_team_stats['average_cards_per_match_pre_match'] + away_team_stats['average_cards_per_match_pre_match']) / 2],
            'home_team_goal_count': [0],  # Preencher com 0 porque é uma partida futura
            'away_team_goal_count': [0],  # Preencher com 0 porque é uma partida futura
            'home_team_encoded': [home_team_encoded],
            'away_team_encoded': [away_team_encoded]
        })

    # Certificar-se de que as colunas do novo conjunto de dados correspondem às do conjunto de treino
    match_data = match_data[X.columns]

    # Imputação e padronização dos dados
    match_data_imputed = imputer.transform(match_data)
    match_data_scaled = scaler.transform(match_data_imputed)

    # Prevendo as probabilidades para cada resultado (vitória da casa, empate, vitória do visitante)
    probabilities = rf.predict_proba(match_data_scaled)[0]

    # Extraindo as probabilidades para cada resultado
    home_win_prob = probabilities[2]
    draw_prob = probabilities[0]
    away_win_prob = probabilities[1]

    # Classificar as probabilidades usando a função categorize_win_probability
    home_team_chance = categorize_win_probability(home_win_prob)
    away_team_chance = categorize_win_probability(away_win_prob)

    # Exibir o resultado formatado com as categorias
    result = {
        'Probabilidade de vitória da casa (%)': home_win_prob * 100,
        'Chance de vitória da casa': home_team_chance,
        'Probabilidade de empate (%)': draw_prob * 100,
        'Probabilidade de vitória do visitante (%)': away_win_prob * 100,
        'Chance de vitória do visitante': away_team_chance
    }


    # Exibir o resultado formatado
    for key, value in result.items():
        print(f"{key}: {value}")

    return result

#### Exemplo de Uso
**Explicação:**
- `Exemplo prático:` Aqui, aplicamos a função de previsão para um jogo hipotético entre dois times. As probabilidades de vitória para ambos os times e a classificação qualitativa são exibidas.

In [ ]:
# Exemplo de uso com times codificados
resultado = predict_match_result_with_averages(home_team_input='Corinthians', away_team_input=17)

#### 3.3 Métricas do modelo
*Explicação:*
Nessa célula abordamos o uso das métricas, isto é, como explicado anteriormente, a avaliação de desempenho das predições. Usamos o submódulo metrics do Scikit-Learn, para conseguir acessar as funções de acurácia, acurácia balanceada, precisão, recall, F1-Score e a Matriz de Confusão. Nesse sentido, ao observar o conjunto de dados, é possível concluir que a grande maioria dos dados são compostos por empates, oque configura na condição de "classes desbalanceadas". Em vista disso, focamos em aumentar as porcentagens de acurácia balanceada e F1-Score, que são duas métricas que são ideais ao enfrentar esse tipo de dados.

In [ ]:
# Avaliando o modelo com as métricas
y_pred = rf.predict(X_test)
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Acurácia Balanceada:", balanced_accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))
print("Matrix de confusão: \n", confusion_matrix(y_test, y_pred))


#### 3.4 Primeiro Modelo Candidato:

O primeiro modelo implementado foi um **Random Forest Classifier** com 100 estimadores (*trees*), e as seguintes etapas de pré-processamento foram realizadas:
- **Imputação de Valores Faltantes**: Foi utilizado o método da média para imputar valores faltantes.
- **Padronização dos Dados**: As características foram padronizadas utilizando o *StandardScaler* para garantir que todas as variáveis tenham uma escala uniforme.
- **Divisão dos Dados**: O conjunto de dados foi dividido em 70% para treino e 30% para teste, com uma semente aleatória fixa para garantir reprodutibilidade.

### Resultados e Discussão:

Para o primeiro modelo, foram usadas as seguintes métricas de avaliação:
1. **Acurácia**: A porcentagem de previsões corretas no conjunto de teste. A acurácia do modelo foi de aproximadamente `xx%`.
   - *Discussão*: Embora a acurácia seja uma métrica importante, ela pode ser enganosa em conjuntos de dados desbalanceados. No caso de jogos de futebol, onde há três resultados possíveis (vitória, empate ou derrota), a acurácia precisa ser interpretada com cuidado.
   
2. **Precision, Recall e F1-Score**:
   - **Precision**: Mede a precisão das previsões de cada classe (se a equipe da casa vence, empate ou a equipe visitante vence).
   - **Recall**: Mede a capacidade do modelo de encontrar todas as instâncias corretas de cada classe.
   - **F1-Score**: Uma média harmônica entre *precision* e *recall*, equilibrando os dois. O F1-Score para a classe de vitória da equipe da casa foi `xx`, para empate foi `xx` e para a vitória do visitante foi `xx`.
   - *Discussão*: Essas métricas fornecem uma visão mais completa do desempenho do modelo do que a acurácia isolada. Por exemplo, um F1-Score mais alto para uma classe específica indica que o modelo está prevendo bem essa classe, equilibrando falsos positivos e falsos negativos.
   
3. **Matriz de Confusão**: A matriz de confusão mostrou a distribuição de previsões corretas e incorretas para cada classe.
   - *Discussão*: A matriz revelou que o modelo estava tendo mais dificuldade em prever empates, sugerindo que mais refinamentos poderiam ser feitos nessa classe para melhorar a precisão. Este é um resultado comum em modelos de futebol, dado que empates são menos frequentes do que vitórias.

### Melhorias Propostas:
Com base nos resultados do primeiro modelo candidato, algumas melhorias foram identificadas:
- **Ajuste de Hiperparâmetros**: Testar diferentes números de estimadores e a profundidade das árvores para encontrar um modelo mais ajustado.
- **Reamostragem das Classes**: Utilizar técnicas para balancear as classes (por exemplo, *oversampling* de empates) pode melhorar a capacidade do modelo de prever empates.
- **Incorporação de Novas Variáveis**: Incluir variáveis que possam capturar outros aspectos importantes do jogo, como o fator casa/fora, lesões de jogadores, entre outros.

## Conclusão:

O primeiro modelo de Random Forest apresentou bons resultados, com alta acurácia e bons valores de F1-Score para a previsão de vitórias e derrotas. No entanto, foi identificado que o modelo apresenta dificuldades na previsão de empates, sugerindo a necessidade de aprimoramentos no balanceamento de classes e no ajuste dos hiperparâmetros.